<a href="https://colab.research.google.com/github/hoainv99/bug_localization/blob/master/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Processing data


In [1]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

def max_length(tensor):
  return max(len(t) for t in tensor)
def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_texts(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,maxlen=300,padding='post')

  return tensor, lang_tokenizer
def load_dataset(path, num_examples=None):
  # creating cleaned input, output pairs
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer


In [2]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import pandas as pd 
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w
# def create_dataset(path, num_examples):
#   lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

#   word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]

#   return zip(*word_pairs)
def create_dataset(path, num_examples):
  data=pd.read_csv(path,encoding='UTF-8')
  lines=[]
  for nl,ct in zip(data["nature_language"],data["code_token"]):
    lines.append([ct,nl])
  word_pairs = [[preprocess_sentence(w) for w in l]  for l in lines]

  return zip(*word_pairs)


## Model

In [3]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))
  def initialize_hidden_state_evaluate(self):
    return tf.zeros((1, self.enc_units))    
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):

    query_with_time_axis = tf.expand_dims(query, 1)
    score = self.V(tf.nn.tanh(
        self.W1(query_with_time_axis) + self.W2(values)))

    attention_weights = tf.nn.softmax(score, axis=1)

    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
    self.attention = BahdanauAttention(self.dec_units)
    
  def call(self, x, hidden, enc_output):
    context_vector, attention_weights = self.attention(hidden, enc_output)
    x = self.embedding(x)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
    output, state = self.gru(x)
    output = tf.reshape(output, (-1, output.shape[2]))
    x = self.fc(output)
    return x, state, attention_weights

## Load data and generate

In [4]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

path_to_file = '/content/drive/My Drive/pj2/dataset.csv'
# Try experimenting with the size of that dataset
num_examples = 10000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)
# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)
print(len(input_tensor))

input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.25)

BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 8
steps_per_epoch_train = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 512
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset_train = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset_train = dataset_train.batch(BATCH_SIZE, drop_remainder=True)



BUFFER_SIZE = len(input_tensor_val)

steps_per_epoch_val = len(input_tensor_val)//BATCH_SIZE

dataset_val = tf.data.Dataset.from_tensor_slices((input_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)
dataset_val = dataset_val.batch(BATCH_SIZE, drop_remainder=True)


8413


## Test model and loss function

In [5]:
example_input_batch, example_target_batch = next(iter(dataset_train))
example_input_batch.shape, example_target_batch.shape
#encode
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)

attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

checkpoint_dir = '/content/drive/My Drive/pj2/checkpoint/checkpoint'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)


## Train model

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden).gpu()

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)
      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss
def val_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)
      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  return batch_loss
EPOCHS = 10
best_loss = 1e+9
for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss_train = 0
  total_loss_val = 0
  for (batch, (inp, targ)) in enumerate(dataset_train.take(steps_per_epoch_train)):
    batch_loss = train_step(inp, targ, enc_hidden)
    
    total_loss_train += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss_train {:.4f} '.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  for (batch, (inp, targ)) in enumerate(dataset_val.take(steps_per_epoch_val)):
    batch_loss = val_step(inp, targ, enc_hidden)
    
    total_loss_val += batch_loss
  # saving (checkpoint) the model every 2 epochs
  total_loss_val = total_loss_val / steps_per_epoch_val
  if best_loss >= total_loss_val:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss_train {:.4f} Loss_val {:.4f}'.format(epoch + 1,
                                      total_loss_train / steps_per_epoch_train,total_loss_val))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

## load checkpoint

In [6]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Evaluate

In [7]:
def evaluate(inp, targ, enc_hidden):
  max_length_targ = 300
  max_length_inp = 300

  attention_plot = np.zeros((max_length_targ, max_length_inp))

  #inp
  inp = preprocess_sentence(inp)

  inputs = [inp_lang.word_index[i] for i in inp.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)
  #targ
  targ = preprocess_sentence(targ)

  targets = [targ_lang.word_index[i] for i in targ.split(' ')]
  targets = tf.keras.preprocessing.sequence.pad_sequences([targets],
                                                         maxlen=max_length_targ,
                                                         padding='post')
  targets = tf.convert_to_tensor(targets)
  loss = 0
  ####
  enc_output, enc_hidden = encoder(inputs, enc_hidden)

  dec_hidden = enc_hidden

  dec_input = tf.expand_dims([targ_lang.word_index['<start>']] , 1)

  # Teacher forcing - feeding the target as the next input
  for t in range(1, targets.shape[1]):
    # passing enc_output to the decoder
    predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

    loss += loss_function(targets[:, t], predictions)

    # using teacher forcing
    dec_input = tf.expand_dims(targets[:, t], 1)

  total_loss = loss/int(targets.shape[1])
  return total_loss

In [8]:
def Score_bug_source(bug,source):
  enc_hidden = encoder.initialize_hidden_state_evaluate()
  score = evaluate(bug, source, enc_hidden)
  return score

In [9]:
import string
def preprocessing(sentence):
  table = str.maketrans(dict.fromkeys(string.punctuation))
  return sentence.translate(table)   

In [10]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/JDT_evaluate.csv")

In [11]:
from sklearn.metrics import classification_report
y_true = []
y_pred = []
threshold = 0.5
scores=[] #array save loss of pair bug reports and source code to test with anothers thresold
for source,target,label in zip(df['nature_language'][:200],df['code_token'][:200],df['label'][:200]):
  try:
    score = Score_bug_source(preprocessing(source),preprocessing(target))
    scores.append(score)
    pred=0
    if score < threshold:
      pred = 1
    y_true.append(label)
    y_pred.append(pred)
  except:
    continue

## Accuaracy


In [12]:
target_names = ['class 0',"class 1"]
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.74      0.81      0.77        21
     class 1       0.75      0.67      0.71        18

    accuracy                           0.74        39
   macro avg       0.74      0.74      0.74        39
weighted avg       0.74      0.74      0.74        39



## MAP & MRR

In [26]:
mrr = []
map = []
# sort
check_0 = 0
check_1 = 0
for i in range(len(scores)):
  inc = 1
  if (y_pred[i] == 0) and (y_true[i] ==0) and (check_0==0):
    mrr.append(1/(i+1))
    check_0 = 1
  if y_pred[i] ==1 and y_true[i] == 1 and check_1==0:
    mrr.append(1/(i+1))
    check_1 = 1
  if check_0==check_1==1:
    break
ap_0 = []
ap_1 = []
for i in range(len(scores)):  
  relevant_0 = 0
  relevant_1 = 0
  length= len(scores[:i])+1
  for j in range(length):
      if y_pred[j] == y_true[j] == 0:
        relevant_0 += 1
      if  y_pred[j] == y_true[j] == 1:
        relevant_1 += 1
  ap_0.append(relevant_0/length)
  ap_1.append(relevant_1/length)

In [27]:
np.mean(mrr)

0.75

In [28]:
map =  (np.mean(ap_0) + np.mean(ap_1))/2

In [29]:
map

0.380719380401289